In [4]:
import pandas as pd

motsJoyeux = pd.read_table("motsJoyeux", sep = ",", header= None)

motsJoyeux = motsJoyeux.transpose()

motsJoyeux = motsJoyeux.rename(columns={0: "word"})

motsJoyeux['polarity'] = 1 

motsJoyeux.head(5)

,word,polarity
0,À BAS PRIX,1
1,À COUPER LE SOUFFLE,1
2,À CROISSANCE RAPIDE,1
3,À ÊTRE VU,1
4,À JUSTE TITRE,1


In [6]:
import pandas as pd

motsNegatifs = pd.read_table("motsNegatifs", sep = ",", header= None)

motsNegatifs = motsNegatifs.transpose()


motsNegatifs = motsNegatifs.rename(columns={0: "word"})

motsNegatifs['polarity'] = -1

motsNegatifs.head(5)

,word,polarity
0,AMERTUNE,-1
1,ABATTEMENT,-1
2,ABSURDITÉ,-1
3,ACCABLEMENT,-1
4,AFFLICTION,-1


In [18]:
FEEL = pd.read_csv("nlp_fr/FEEL.csv", sep = ";")

FEEL = FEEL.drop(['id', 'joy', "fear", "sadness", "anger", "surprise", "disgust"], axis=1)

FEEL = FEEL.replace({'polarity': {"positive": 1, "negative": -1}})

FEEL["polarity"].astype(int)


result = pd.concat([FEEL, motsJoyeux, motsNegatifs],  join='outer')
result.drop(result.index[155])
result.head(5)


,word,polarity
0,à ce endroit là,1
1,à le hâte,-1
2,à part,-1
3,à pic,-1
4,à rallonge,-1


In [23]:
import pandas as pd
from lxml import etree
doc = etree.parse("nlp_fr/fr-sentiment.xml")

root = doc.getroot()
word = doc.find('word')
words  = [word.get('form') for word in root.iter()]
polarity = [word.get('polarity') for word in root.iter()]


polarity = pd.Series(polarity).to_frame()
words  = pd.Series(words).to_frame()
total = pd.concat([words, polarity], axis = 1)

total.columns = ["word","polarity"]
total = total.dropna()
total.head(5)

total["polarity"] = pd.to_numeric(total["polarity"], errors='coerce')


total["polarity1"] = ""
total["polarity1"][total.polarity >0] = 1
total["polarity1"][total.polarity <0] = -1
total = total[total.polarity != 0]

total = total.drop(["polarity"], axis=1)
total = total.rename(columns={"polarity1": "polarity"})
total.head(5)


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,word,polarity
1,abandonné,-1
2,abandonnée,-1
3,abandonnées,-1
4,abandonnés,-1
5,abasourdi,1


In [21]:
result1 = pd.concat([result, total],  join='outer')
result1.dtypes
result1 = result1[result1.polarity != 0]

result1.head(5)





,word,polarity
0,à ce endroit là,1
1,à le hâte,-1
2,à part,-1
3,à pic,-1
4,à rallonge,-1


In [22]:
import numpy as np
pd.options.display.max_colwidth = 300
reddit = pd.read_table("french-reddit-discussion/final_SPF_2.xml",sep="'<utt'|'/utt>'",header=None, engine='python', encoding="utf-8")
reddit.columns = ["brut"]
conv = reddit[reddit.brut.str.startswith("<utt",na=np.nan)]

conv['brut'] = conv['brut'].str.replace('<utt.*">', '')
conv['brut'] = conv['brut'].str.replace('</utt>', '')


conv.head(5)


                

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,brut
39,"Ironie : l'article disant qu'on est plus capable de se concentrer plus de 2 pages, en fait 6 !"
41,"Moi-même, j'ai dû me forcer pour arriver jusqu'à la fin. Et pourtant ça ne devient pas moins intéressant. Ca fout les jetons... :)"
44,Service qui sera rendu au contribuable pour la modique some de 70 Millions par an.
45,Eeeeh oui ! 70 millions pour une loi qui aura pour effet de .
48,Est-ce qu'elle a vraiment commis des crimes qui méritent **soixante** ans de prison? J'en doute...


In [24]:
conv = conv.reset_index(drop=True)
result1 = result1.reset_index(drop=True)
conv.brut = conv.brut.str.lower()
conv["score"]= 0

In [8]:
# noyau du plugin : 2 boucles 

# partition = conv[0:2]
# for i in range(len(partition)):
    #for j in range(len(result1)):
        #for k in range (len(partition.loc[i].brut.split())) : 
            #if(conv.loc[i].brut.split()== result1.word[j]):
                #if(result1.polarity[j] == "positive"):
                    #conv.score[i] += 1
                #else:
                    #conv.score[i] -=1


In [9]:
#import time 
#starttime = time.time()

#test = conv.iloc[2]

#score = 0
#for i in range(len(test)):
    #for j in range(len(result1)):
        #if(test.brut.split()[i]== result1.word[j]):
            #if(result1.polarity[j] == "positive"):
                #score += 1
            #else:
                #score -=1



#print(score)
#print("%s secondes" %(time.time()-starttime))



In [25]:
conv.dtypes

brut     object
score     int64
dtype: object

In [11]:
#import nltk
#from nltk.corpus import stopwords
#nltk.download('stopwords')

#clean_tokens = conv[:]
 
#stopwords = set(stopwords.words('french'))
 
#for token in partition['tokenized']:
 
    #if token in stopwords:
 
        #clean_tokens.remove(token)

#print(clean_tokens)

In [12]:
#from nltk.tokenize import sent_tokenize, word_tokenize
#import nltk
#from nltk.corpus import stopwords
#nltk.download('stopwords')

#stopWords = stopwords.words('french')

#words = word_tokenize(conv["brut"])

#wordsFiltered = []

#for w in partition['tokenized']:
   #if w not in stopWords:
     #wordsFiltered.append(w)

#print (wordsFiltered)

In [26]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

partition = conv[0:1500]
partition[pd.DataFrame(partition.brut.str.split().tolist()).isin(FEEL.word.tolist()).any(1)]

partition['tokenized'] = partition.apply(lambda row: nltk.word_tokenize(row['brut']), axis=1)

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [27]:
scorable_part = partition[pd.DataFrame(partition.tokenized.tolist()).isin(FEEL.word.tolist()).any(1)]
scorable_part

,brut,score,tokenized
0,"ironie : l'article disant qu'on est plus capable de se concentrer plus de 2 pages, en fait 6 !",0,"[ironie, :, l'article, disant, qu'on, est, plus, capable, de, se, concentrer, plus, de, 2, pages, ,, en, fait, 6, !]"
1,"moi-même, j'ai dû me forcer pour arriver jusqu'à la fin. et pourtant ça ne devient pas moins intéressant. ca fout les jetons... :)",0,"[moi-même, ,, j'ai, dû, me, forcer, pour, arriver, jusqu'à, la, fin, ., et, pourtant, ça, ne, devient, pas, moins, intéressant, ., ca, fout, les, jetons, ..., :, )]"
2,service qui sera rendu au contribuable pour la modique some de 70 millions par an.,0,"[service, qui, sera, rendu, au, contribuable, pour, la, modique, some, de, 70, millions, par, an, .]"
3,eeeeh oui ! 70 millions pour une loi qui aura pour effet de .,0,"[eeeeh, oui, !, 70, millions, pour, une, loi, qui, aura, pour, effet, de, .]"
4,est-ce qu'elle a vraiment commis des crimes qui méritent **soixante** ans de prison? j'en doute...,0,"[est-ce, qu'elle, a, vraiment, commis, des, crimes, qui, méritent, **soixante**, ans, de, prison, ?, j'en, doute, ...]"
5,"elle a été jugée par deux fois, d'aider et de participer à l'enlèvement. elle l'a fait plusieurs fois en fonction de ses victimes qui se sont identifiés. les membres de la bande, elle a travaillé avec elle et a également identifié avoué. elle a été jugé deux fois.",0,"[elle, a, été, jugée, par, deux, fois, ,, d'aider, et, de, participer, à, l'enlèvement, ., elle, l'a, fait, plusieurs, fois, en, fonction, de, ses, victimes, qui, se, sont, identifiés, ., les, membres, de, la, bande, ,, elle, a, travaillé, avec, elle, et, a, également, identifié, avoué, ., elle,..."
7,"si ils ne sortent pas des idées pertinentes d'ici un an, ils sont morts. la seul chose que produit le ps depuis plusieurs années maintenant, c'est:",0,"[si, ils, ne, sortent, pas, des, idées, pertinentes, d'ici, un, an, ,, ils, sont, morts, ., la, seul, chose, que, produit, le, ps, depuis, plusieurs, années, maintenant, ,, c'est, :]"
8,"soit sego, soit la mort: elle prend trop de place, ils ne pourront jamais s'en débarrasser donc soit donner le ps à sego soit le hara-kiri",0,"[soit, sego, ,, soit, la, mort, :, elle, prend, trop, de, place, ,, ils, ne, pourront, jamais, s'en, débarrasser, donc, soit, donner, le, ps, à, sego, soit, le, hara-kiri]"
9,"donner le ps à une people sans idées? quelle drôle d'idée! non, il faut arrêter les grandes déclarations et commencer à contribuer par des idées au débat national. evidemment, si les médias relayaient ce genre d'initiatives plutôt que les frasques de ségolène, ça aiderait...",0,"[donner, le, ps, à, une, people, sans, idées, ?, quelle, drôle, d'idée, !, non, ,, il, faut, arrêter, les, grandes, déclarations, et, commencer, à, contribuer, par, des, idées, au, débat, national, ., evidemment, ,, si, les, médias, relayaient, ce, genre, d'initiatives, plutôt, que, les, frasque..."
10,à quand la taxe tobin?,0,"[à, quand, la, taxe, tobin, ?]"


In [28]:
from nltk.stem import SnowballStemmer
fr_stemmer = SnowballStemmer('french')
FEEL['stemmed'] = FEEL.word.apply(fr_stemmer.stem)


In [29]:
FEEL = FEEL.drop_duplicates("stemmed")
FEEL.head(5)

,word,polarity,stemmed
0,à ce endroit là,1,à ce endroit là
1,à le hâte,-1,à le hât
2,à part,-1,à part
3,à pic,-1,à pic
4,à rallonge,-1,à rallong


In [30]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = set(stopwords.words('french'))
partition['clean_tokens'] = partition['tokenized'].apply(lambda x: [i for i in x if i not in stop] )
partition.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,brut,score,tokenized,clean_tokens
0,"ironie : l'article disant qu'on est plus capable de se concentrer plus de 2 pages, en fait 6 !",0,"[ironie, :, l'article, disant, qu'on, est, plus, capable, de, se, concentrer, plus, de, 2, pages, ,, en, fait, 6, !]","[ironie, :, l'article, disant, qu'on, plus, capable, concentrer, plus, 2, pages, ,, fait, 6, !]"
1,"moi-même, j'ai dû me forcer pour arriver jusqu'à la fin. et pourtant ça ne devient pas moins intéressant. ca fout les jetons... :)",0,"[moi-même, ,, j'ai, dû, me, forcer, pour, arriver, jusqu'à, la, fin, ., et, pourtant, ça, ne, devient, pas, moins, intéressant, ., ca, fout, les, jetons, ..., :, )]","[moi-même, ,, j'ai, dû, forcer, arriver, jusqu'à, fin, ., pourtant, ça, devient, moins, intéressant, ., ca, fout, les, jetons, ..., :, )]"
2,service qui sera rendu au contribuable pour la modique some de 70 millions par an.,0,"[service, qui, sera, rendu, au, contribuable, pour, la, modique, some, de, 70, millions, par, an, .]","[service, rendu, contribuable, modique, some, 70, millions, an, .]"
3,eeeeh oui ! 70 millions pour une loi qui aura pour effet de .,0,"[eeeeh, oui, !, 70, millions, pour, une, loi, qui, aura, pour, effet, de, .]","[eeeeh, oui, !, 70, millions, loi, effet, .]"
4,est-ce qu'elle a vraiment commis des crimes qui méritent **soixante** ans de prison? j'en doute...,0,"[est-ce, qu'elle, a, vraiment, commis, des, crimes, qui, méritent, **soixante**, ans, de, prison, ?, j'en, doute, ...]","[est-ce, qu'elle, a, vraiment, commis, crimes, méritent, **soixante**, ans, prison, ?, j'en, doute, ...]"


In [39]:
fr_stemmer = SnowballStemmer('french')
partition['stemmed'] = partition.clean_tokens.apply(lambda x: [fr_stemmer.stem(y) for y in x])

partition['stemmed']


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0                                                                                                                                                                                                                              [iron, :, l'articl, dis, qu'on, plus, capabl, concentr, plus, 2, pag, ,, fait, 6, !]
1                                                                                                                                                                                       [moi-mêm, ,, j'ai, dû, forc, arriv, jusqu'à, fin, ., pourt, ça, devient, moin, intéress, ., ca, fout, le, jeton, ..., :, )]
2                                                                                                                                                                                                                                                        [servic, rendu, contribu, modiqu, som, 70, million, an, .]
3                                                                           

In [33]:
# count vectorizer



In [ ]:
def scoring(list_):
    res = 0 
    for i in range(len(partition.stemmed])):
        for j in range(len(FEEL.stemmed)):
            if partition.stemmed[i] == FEEL.stemmed[j]:
                res = res + FEEL.polarity[j]
    return res